In [4]:
!pip install PyPDF2

Looking in indexes: http://mirrors.aliyun.com/pypi/simple, http://mirrors.aliyun.com/pypi/simple/, http://pypi.doubanio.com/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.5 MB/s eta 0:00:00a 0:00:01


In [17]:
import PyPDF2

def extract_pdf_toc(pdf_path):
    # 打开 PDF 文件
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # 获取目录（书签）
        toc = reader.outline
        
        # 递归解析目录树
        def parse_toc(toc, level=0):
            for item in toc:
                if isinstance(item, list):
                    parse_toc(item, level + 1)
                else:
                    title = item.title
                    page_num = reader.get_destination_page_number(item) + 1  # 获取页码
                    indent = ' ' * level * 4
                    print(f"{indent}{title} (Page {page_num})")
        
        parse_toc(toc)



# 指定PDF文件的路径
pdf_path = '/home/zhuxy/OSAutoFormalizer/KGConstruct/db/Abstract Formal Specification of the seL4ARMv6 API.pdf'
extract_pdf_toc(pdf_path)

Introduction (Page 11)
    The seL4 Microkernel (Page 11)
        Kernel Services (Page 11)
        Capability-based Access Control (Page 13)
        Kernel Objects (Page 13)
        Kernel Memory Allocation (Page 18)
    Summary (Page 19)
    Theory Dependencies (Page 20)
Nondeterministic State Monad with Failure (Page 21)
    The Monad (Page 21)
        Nondeterminism (Page 22)
        Failure (Page 22)
        Generic functions on top of the state monad (Page 23)
        The Monad Laws (Page 24)
    Adding Exceptions (Page 24)
        Monad Laws for the Exception Monad (Page 25)
    Syntax (Page 26)
        Syntax for the Nondeterministic State Monad (Page 26)
        Syntax for the Exception Monad (Page 27)
    Library of Monadic Functions and Combinators (Page 27)
    Catching and Handling Exceptions (Page 29)
        Loops (Page 30)
    Hoare Logic (Page 31)
        Validity (Page 31)
        Determinism (Page 32)
        Non-Failure (Page 32)
    Basic exception reasoning (Page 

In [1]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
with open('/home/zhuxy/OSAutoFormalizer/KGConstruct/md/Abstract Formal Specification of the seL4ARMv6 API.md', 'r', encoding='utf-8') as file:
    markdown_text = file.read()

# 创建MarkdownHeaderTextSplitter实例，并指定分割级别
splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
    ("#", "Paragraph"),
    ("##", "Section"),
    ("###", "SubSection")
])

# 使用split_text方法对Markdown内容进行分割
sections = splitter.split_text(markdown_text)

unique_section_ = set()

for section in sections:
    section_title = section.metadata.get("Section", "")
    if section_title not in unique_section_:
        unique_section_.add(section_title)

import re
pattern = r'^(\d+(\.\d+)*)\s'
cleaned_sections = [re.sub(pattern, '', section) for section in unique_section_]
print(cleaned_sections)
print(len(cleaned_sections))

['', 'Sending Fault Messages', 'Determinism', 'Kernel Objects', 'Getting And Setting The Message Info Register.', 'Kernel Services', 'Cnode', 'Synchronous Message Transfers', 'General Object Access', 'Memory Clearance', 'Error And Fault Messages', 'Toplevel Invocation Decode.', 'Wrapping And Lifting Machine Operations', 'Tcbs', 'Machine State', 'System Call Entry Point', 'Summary', 'Kernel State', 'Encodings', 'Interrupt Objects', 'User Monad', 'Syntax For The Exception Monad', 'Ipc Capability Transfers', 'User Context', 'Capability-Based Access Control', 'Generic Functions On Top Of The State Monad', 'Re-Using Memory', 'Basic Capability Manipulation', 'The Monad', 'The Monad Laws', 'Threads', 'Capability Access', 'Resolving Capability References', 'Nondeterminism', 'Cnodes', 'Monad Laws For The Exception Monad', 'Thread Message Formats', 'The Operations', 'Failure', 'Activating Threads', 'Irq', 'Fault Handling', 'Library Of Monadic Functions And Combinators', 'Untyped', 'Architecture-

In [2]:
sections

[Document(page_content='This document is the text version of the abstract, formal Isabelle/HOL specification of the seL4 microkernel. It is intended to give a precise, operational definition of the seL4 microkernel on the ARMv6 architecture. The document contains a short overview, followed by text generated from the formal Isabelle/HOL definitions. This document is not a tutorial or user manual and is not intended to be read as such. Please see the bundled user manual for a higher-level introduction to the kernel.', metadata={'Paragraph': 'Abstract'}),
 Document(page_content='The seL4 microkernel is an operating system kernel designed to be a secure, safe, and reliable foundation for systems in a wide variety of application domains. As a microkernel, seL4 provides a minimal number of services to applications. This small number of services directly translates to a small implementation of approximately 8700 lines of C code, which has allowed the kernel to be formally proven in the Isabel

In [3]:
import os
from neo4j import GraphDatabase
import PyPDF2

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def write_queries_to_neo4j(query, valid_queries):
    def add_query_and_rel(tx, query, sub_query):
        tx.run("""
            MERGE (q:Query {name: $query})
            MERGE (sq:Query {name: $sub_query})
            MERGE (q)-[:SPECIFY]->(sq)
            """, parameters={"query": query, "sub_query": sub_query})
    
    with driver.session() as session:
        for sub_query in valid_queries:
            # 跳过自己指向自己
            if query == sub_query:
                continue
            session.write_transaction(add_query_and_rel, query, sub_query)

def extract_pdf_toc_and_insert_to_neo4j(pdf_path, max_level=3):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        toc = reader.outline
        
        def parse_and_insert(toc, parent="seL4", level=0):
            if level >= max_level:
                return
            
            current_level_titles = []
            excluded_titles = ["Introduction", "Glossary","The seL4 Microkernel","Kernel Services","Capability-based Access Control","Kernel Objects","Kernel Memory Allocation","Summary","Theory Dependencies"]
            
            for item in toc:
                if isinstance(item, list):
                    # 递归处理子目录，前提是当前层级还没达到最大层级
                    parse_and_insert(item, current_level_titles[-1] if current_level_titles else parent, level + 1)
                else:
                    title = item.title
                    # 跳过排除的标题及其子标题
                    if any(excluded in title for excluded in excluded_titles):
                        continue
                    current_level_titles.append(title)
                    # 将当前标题与父级标题插入Neo4j
                    write_queries_to_neo4j(parent, [title])
            
        parse_and_insert(toc)


pdf_path = '/home/zhuxy/OSAutoFormalizer/KGConstruct/db/Abstract Formal Specification of the seL4ARMv6 API.pdf'
extract_pdf_toc_and_insert_to_neo4j(pdf_path, max_level=5)



/tmp/ipykernel_1581351/3456571189.py:26: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_query_and_rel, query, sub_query)


In [6]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE", "seL4")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def write_queries_to_neo4j(query, valid_queries):
    def add_query_and_rel(tx, query, sub_query):
        tx.run("""
            MERGE (q:Query {name: $query})
            MERGE (sq:Query {name: $sub_query})
            MERGE (q)-[:SPECIFY]->(sq)
            """, parameters={"query": query, "sub_query": sub_query})
    
    with driver.session() as session:
        for sub_query in valid_queries:
            # 跳过自己指向自己
            if query == sub_query:
                continue
            session.write_transaction(add_query_and_rel, query, sub_query)

In [28]:
import json
capDL =  [
    'Asid_D',
    'CNode_D',
    'CSpace_D',
    'Decode_D',
    'Endpoint_D',
    'Intents_D',
    'Interrupt_D',
    'Invocations_D',
    'KHeap_D',
    'Monads_D',
    'PageTable_D',
    'PageTableUnmap_D',
    'Schedule_D',
    'Syscall_D',
    'Tcb_D',
    'Types_D',
    'Untyped_D'
]
with open('./cleaned_output3_unique.jsonl','r') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        if data['title'] in capDL:
            with open('test.jsonl', 'a') as f:
                f.write(line)
        else:
            with open('train.jsonl', 'a') as f:
                f.write(line)

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter, CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.schema import Document
import glob

local_llm = "llama3.1:70b"

# LLM
llm = ChatOllama(model=local_llm, temperature=0.8, num_ctx=12288)

# 用于存储所有的 content
all_contents = []

file_paths = glob.glob("./md2/*.md")

for file_path in file_paths:
    # 读取Markdown文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_text = file.read()

    # 创建MarkdownHeaderTextSplitter实例，并指定分割级别
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "Paragraph"),
        ("##", "Section")
    ])

    # 使用split_text方法对Markdown内容进行分割
    sections = splitter.split_text(markdown_text)

    for section in sections:
        section_title = section.metadata.get("Section", "")
        paragraph_title = section.metadata.get("Paragraph", "")

        # 为每个section创建新的ConversationBufferMemory
        memory = ConversationBufferMemory()
        conversation = ConversationChain(
            llm=llm,
            memory=memory
        )

        # 使用 CharacterTextSplitter 对 section 进一步分割
        char_splitter = CharacterTextSplitter(chunk_size=1200)
        contents = char_splitter.split_text(section.page_content)

        for idx, content in enumerate(contents):
            # 第一轮对话，包含系统提示信息和段落/章节信息
            if idx == 0:
                summary_prompt = f"You are a helpful assistant in summarizing Operating System documentation. " \
                                 f"Please summarize the following context in concise language with up to 10 sentences. Please answer with the summary only, no introductory or concluding statements." \
                                 f"Context: {paragraph_title} {section_title}\n{content}"
            else:
                summary_prompt = f"Context: {content}"

            # 使用 ConversationChain 生成摘要
            summary = conversation.run(summary_prompt)

            # 创建 Document 对象，包含摘要和原始内容
            document = Document(
                page_content=summary,
                metadata={"content":content ,"section":section_title,"paragraph":paragraph_title}
            )

            # 将 Document 添加到 all_contents 列表中
            all_contents.append(document)

            print(f"Summary: {summary}")


In [ ]:
import json

# 将 all_contents 转换为 JSONL 格式并写入到文件
with open('./documents.jsonl', 'a', encoding='utf-8') as f:
    for doc in all_contents:
        # 将 Document 对象转换为字典
        doc_dict = {
            "summary": doc.page_content,
            "metadata": doc.metadata
        }
        # 将字典转换为 JSON 字符串，并写入到文件，每个对象占一行
        f.write(json.dumps(doc_dict) + '\n')

print("JSONL 文件已生成。")

In [ ]:
import glob
from langchain_community.document_loaders import PDFMinerLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

file_paths = glob.glob("./information-security/*.pdf")


# 初始化一个列表来存储所有文件的分块内容
all_chunks = []

# 遍历每个PDF文件路径
for file_path in file_paths:
    print(file_path)
    # 创建PDFMinerLoader对象
    loader = PDFMinerLoader(file_path, concatenate_pages=True)
    
    # 加载PDF文件内容
    data = loader.load()
    
    # 创建CharacterTextSplitter对象
    text_splitter = CharacterTextSplitter(chunk_size=800)
    
    # 分割加载的文档内容
    chunks = text_splitter.split_documents(data)
    
    # 将分割的内容添加到总列表中
    all_chunks.extend(chunks)

vectordb = Chroma.from_documents(
    documents=all_chunks,
    collection_name='seL4_4',
    embedding=hf,
    persist_directory="../vectordb"
)